# Tensorflow Tutorial

In this file, I will follow the Tensorflow official tutorial to learn Tensorflow. In the first place, https://www.tensorflow.org/get_started/ and then https://www.tensorflow.org/tutorials/. In the meantime, I probably will have my own small projects using Tensorflow. 

## Getting Started

In [1]:
import tensorflow as tf

The computational graph: Tensorflow Core programs consist of two discrete sections
1) Building the comptational graph
2) Ruing the computational graph
Computational graph is a series Tensorflow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. 

In [3]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
node3 = tf.constant(5.0)
print node1, node2, node3

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32) Tensor("Const_4:0", shape=(), dtype=float32)


To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the Tensorflow runtime. Create a Session object, and invoke its run method to run enough of the computational graph to evaluate. 

In [5]:
session = tf.Session()
print session.run(node1 + node2)
print session.run([node1, node2])

7.0
[3.0, 4.0]


We can build more complicated computation by combining tensor nodes with operations.

In [7]:
node4 = tf.add(node1, node2)
print "node4:", node4
print "session.run(node4)", session.run(node4)

node4: Tensor("Add:0", shape=(), dtype=float32)
session.run(node4) 7.0
